In [1]:
# This section of code is to call an API to provide Top20 Songs in the US 
# and by US Cities and save the charts into 2 panda dataframes

#import libraries 
import requests 
import pandas as pd
import time 


In [2]:
#Create two data frames that will be merged later 

dfUSTop20 = pd.DataFrame(columns=["Song Name","Artist","Rank in USTop20 Chart"])
dfCityTop20 = pd.DataFrame(columns=["Song Name","Artist"])
dfCityTop20

,Song Name,Artist


In [3]:
#Call API For list of countries and its cities

USCitiesURL= "https://shazam-core.p.rapidapi.com/v1/frame/cities"

headers = {
    'x-rapidapi-host': "shazam-core.p.rapidapi.com",
    'x-rapidapi-key': "e6461c6a22mshc27d1e6ff04ffdbp1f4e28jsna11da573e746"
    }

USCitiesResponse = requests.request("GET", USCitiesURL, headers = headers).json()
#USCitiesResponse

In [4]:
#Extract all cities from the US and put each list containing CityID, CityName into a list 
USCities = []
CityIndex = 0
NumberofCities = 202
while CityIndex != NumberofCities:
    CityIDName = []
    CityIDName.append(USCitiesResponse[9]['cities'][CityIndex]['id'])
    CityIDName.append(USCitiesResponse[9]['cities'][CityIndex]['name'])
    USCities.append(CityIDName)
    CityIndex = CityIndex + 1

USCities[0:5]



[['5128581', 'New York City'],
 ['5368361', 'Los Angeles'],
 ['4887398', 'Chicago'],
 ['4699066', 'Houston'],
 ['4180439', 'Atlanta']]

In [5]:
# API Call for Top20 Songs for All cities 
CityChartURL = "https://shazam-core.p.rapidapi.com/v1/charts/city"


headers = {
    'x-rapidapi-host': "shazam-core.p.rapidapi.com",
    'x-rapidapi-key': "e6461c6a22mshc27d1e6ff04ffdbp1f4e28jsna11da573e746"
    }
for CityIndex in range(0, 20):
    dfCityTop20[USCities[CityIndex][1]] = None 
    
dfCityTop20


,Song Name,Artist,New York City,Los Angeles,Chicago,Houston,Atlanta,Dallas,Miami,Phoenix,...,Philadelphia,Boston,Anaheim,Denver,Las Vegas,San Diego,Detroit,Seattle,San Antonio,Orlando


In [6]:
# A Helper Function to fill in Song Rankings in the dfCityTop20 dataframe 

def updateSongRank(SongListResponse, SongIndex, dfCityTop20, SongLimit):
    
    #Append top 20 songs to the CityTop20 Dataframe 
    SongRank = 0
    while SongRank < SongLimit: 
        SongName = SongListResponse[SongRank]['title']
        SongArtist = SongListResponse[SongRank]['subtitle']
        
        # append new song/artist to dataframe if not already inside 
        if SongName not in dfCityTop20['Song Name'].unique():
            dfCityTop20 = dfCityTop20.append({"Song Name": SongName, "Artist": SongArtist}, ignore_index = True)
            global SongIndexCounter
            SongIndex.update({SongName: SongIndexCounter})  
            SongIndexCounter += 1
            #print(SongName)
            
        # fill in the value (rank) for the song for the specific city if not the same as previous day
        # update rank for next song
        if (dfCityTop20.at[SongIndex.get(SongName), CityName] != SongRank+1):
            dfCityTop20.at[SongIndex.get(SongName), CityName] = SongRank+1
        SongRank += 1
        
    #print(SongIndex)
    #Reset song rank
    return dfCityTop20

In [7]:
# Fill in the CityTop20 Dataframe for first 20 cities in the list 
# Only use first 20 cities to limit the number of API Calls for this assignment 

SongLimit = 20
CityIndex = 0

# Keeps track of the song and its index in the dfCityTop20 dataframe 
SongIndex = {}
SongIndexCounter = 0

# Loop through each city 
for CityIndex in range(0, 20): 
    
    # extract the cityID and City Name for this city
    CityID = USCities[CityIndex][0]
    CityName = USCities[CityIndex][1]
        
    # API Call to get the top20 Shazam Chart for the city and call helper function to fill in song rankings  
    querystring = {"city_id":CityID,"limit": SongLimit}
    SongListResponse= requests.request("GET", CityChartURL, headers=headers, params=querystring).json()
    dfCityTop20 = updateSongRank(SongListResponse, SongIndex, dfCityTop20, SongLimit)
    
    #update City Index 
    CityIndex = CityIndex + 1

dfCityTop20

,Song Name,Artist,New York City,Los Angeles,Chicago,Houston,Atlanta,Dallas,Miami,Phoenix,...,Philadelphia,Boston,Anaheim,Denver,Las Vegas,San Diego,Detroit,Seattle,San Antonio,Orlando
0,love nwantiti [Remix],CKay Feat. Axel & Dj Yo!,1,NaN,NaN,NaN,8,NaN,12,13,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,Cold Heart (PNAU Remix),Elton John & Dua Lipa,2,2,2,5,15,3,2,1,...,1,2,3,2,1,3,2,2,2,1
2,Chosen,Blxst & Tyga Feat. Ty Dolla $ign,3,NaN,NaN,NaN,18,NaN,NaN,NaN,...,10,11,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN
3,I Hate U,SZA,4,NaN,NaN,NaN,19,NaN,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Need To Know,Doja Cat,5,7,19,17,NaN,19,13,10,...,13,NaN,12,19,NaN,13,19,12,11,15
5,Pepas,Farruko,6,3,NaN,7,NaN,14,3,3,...,NaN,7,4,NaN,8,5,NaN,NaN,3,2
6,Ameno Amapiano (Remix),Goya Menor & Nektunez,7,NaN,NaN,NaN,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Smokin Out The Window,"Bruno Mars, Anderson .Paak & Silk Sonic",8,NaN,12,9,9,12,14,17,...,7,20,NaN,NaN,18,NaN,16,19,14,14
8,We Set The Trends,Jim Jones & Migos,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Big Energy,Latto,10,1,NaN,12,10,6,17,4,...,11,3,1,6,2,2,4,9,4,NaN


In [8]:
#Call for the Top20 Songs from the US Chart

USChartURL = "https://shazam-core.p.rapidapi.com/v1/charts/country"

querystring = {"country_code":"US","limit":"50"}

headers = {
    'x-rapidapi-host': "shazam-core.p.rapidapi.com",
    'x-rapidapi-key': "e6461c6a22mshc27d1e6ff04ffdbp1f4e28jsna11da573e746"
    }

USChartResponse = requests.request("GET", USChartURL, headers=headers, params=querystring).json()

#USChartResponse


In [9]:
# Fill in the USTop20 Datframe with song/artist/ranking 

SongIndex = 0
while(SongIndex < 20):
    SongName = USChartResponse[SongIndex]['title']
    SongArtist = USChartResponse[SongIndex]['subtitle']
    #print(SongName, SongArtist, SongIndex)
    dfUSTop20 = dfUSTop20.append({"Song Name": SongName, "Artist": SongArtist, "Rank in USTop20 Chart": SongIndex+1}, ignore_index=True)
    SongIndex += 1

dfUSTop20

,Song Name,Artist,Rank in USTop20 Chart
0,The Cowboy In Me (Yellowstone Edition),Tim McGraw,1
1,Cold Heart (PNAU Remix),Elton John & Dua Lipa,2
2,Big Energy,Latto,3
3,Super Gremlin,Kodak Black,4
4,Shivers,Ed Sheeran,5
5,Scorpio,Moneybagg Yo,6
6,Pope Is A Rockstar,SALES,7
7,abcdefu,GAYLE,8
8,Smokin Out The Window,"Bruno Mars, Anderson .Paak & Silk Sonic",9
9,Heat Waves,Glass Animals,10


In [10]:
#Export CityTop20 Dataframe as a CSV 
dfCityTop20.to_csv('/Users/zacharywong/github/zacharywong2023/AtlanticRecords/Results/Top20byUSCity.csv', index=False)
dfCityTop20.head()


,Song Name,Artist,New York City,Los Angeles,Chicago,Houston,Atlanta,Dallas,Miami,Phoenix,...,Philadelphia,Boston,Anaheim,Denver,Las Vegas,San Diego,Detroit,Seattle,San Antonio,Orlando
0,love nwantiti [Remix],CKay Feat. Axel & Dj Yo!,1,NaN,NaN,NaN,8,NaN,12,13,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,Cold Heart (PNAU Remix),Elton John & Dua Lipa,2,2,2,5,15,3,2,1,...,1,2,3,2,1,3,2,2,2,1
2,Chosen,Blxst & Tyga Feat. Ty Dolla $ign,3,NaN,NaN,NaN,18,NaN,NaN,NaN,...,10,11,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN
3,I Hate U,SZA,4,NaN,NaN,NaN,19,NaN,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Need To Know,Doja Cat,5,7,19,17,NaN,19,13,10,...,13,NaN,12,19,NaN,13,19,12,11,15


In [11]:
#Export USTop20 Dataframe as a CSV 
dfUSTop20.to_csv('/Users/zacharywong/github/zacharywong2023/AtlanticRecords/Results/Top20inUS.csv', index=False)
dfUSTop20

,Song Name,Artist,Rank in USTop20 Chart
0,The Cowboy In Me (Yellowstone Edition),Tim McGraw,1
1,Cold Heart (PNAU Remix),Elton John & Dua Lipa,2
2,Big Energy,Latto,3
3,Super Gremlin,Kodak Black,4
4,Shivers,Ed Sheeran,5
5,Scorpio,Moneybagg Yo,6
6,Pope Is A Rockstar,SALES,7
7,abcdefu,GAYLE,8
8,Smokin Out The Window,"Bruno Mars, Anderson .Paak & Silk Sonic",9
9,Heat Waves,Glass Animals,10
